## IMPORT LIBRARY

In [ ]:
import tensorflow as tf
import numpy
import seaborn as sns
from keras.layers import Flatten,Dense
from keras_preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

## VISUALISASI DATASET

In [ ]:
class_names=os.listdir("Train")
print(f"classes: {','.join(class_names)}")
folders = os.listdir('Train')
list=[]
for folder in folders:
    print(f"{folder} : {len(os.listdir(f'Train/{folder}'))}")
    list.append(len(os.listdir(f'Train/{folder}')))
plt.pie(list, labels=class_names, startangle=90,autopct=lambda p: '{:.2f}%({:.0f})'.format(p,(p/100)*sum(list)))
plt.show()

In [ ]:
for i,folder in enumerate(folders):
    plt.subplot(3, 8, i + 1)
    plt.title(folder)
    plt.axis('off')
    if ((os.listdir('Train//'+folder)[0])=='.ipynb_checkpoints'):
        plt.imshow(plt.imread(f"Train//{folder}//{os.listdir('Train//'+folder)[2]}"),cmap="gray")
        pass
    else:
        plt.imshow(plt.imread(f"Train//{folder}//{os.listdir('Train//'+folder)[0]}"),cmap="gray")
plt.show()

### AUGMENTASI <a name="AUGMENTASI">

In [ ]:
def custom_augmentation(np_tensor):
    def gaussian_noise(np_tensor):
        mean = 0
        var = numpy.random.randint(1, 26)
        noise = numpy.random.normal(mean,var**0.5,np_tensor.shape)
        return numpy.clip(np_tensor + noise, 0, 255).astype('float')
    augmnted_tensor = gaussian_noise(np_tensor)
    return numpy.array(augmnted_tensor)
img_datagen=ImageDataGenerator(
            preprocessing_function=custom_augmentation,
            rescale = 1.0/255.,
            rotation_range=20,horizontal_flip=False,
            validation_split=0.3,fill_mode='nearest')